# Úkoly:
1. Pomocí Numpy a Numby napište maticové násobení dvou čtvercových matic.
    - Výsledek ukládejte do předem vytvořené matice C, tedy funkce bude mít tvar `def matrix_multiplication(A, B, C)`.
    - Zakompilujte funkci a na maticích o velikosti 1000x1000 změřte čas výpočtu maticového násobení.
    - Porovnejte výsledek s výsledkem z Numpy.


In [13]:
import numpy as np
from numba import jit, prange

def matrix_multiplication(A, B, C):
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]

matrix_multiplication = jit(matrix_multiplication)

In [15]:
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)
C2 = A @ B

In [20]:
C = np.zeros((1000, 1000))
matrix_multiplication(A, B, C)
print(np.allclose(C, C2))

True



2. Upravte kód tak, aby výpočet probíhal v tzv. Chunk módu. To znamená, že výpočet bude rozdělen do menších bloků, které budou postupně počítány. Výsledek bude stejný jako v bodě 1, ale výpočet bude probíhat v menších blocích.
    - výsledný kód bude mít 6 vnořených for cyklů 
        - nejprve 3 přes chunky v každé z OS (i,j,k)
        - ve vnitř 3 cykly přes prvky v každém chunku
    - zakompilujte, očasujte, a porovnejte s výsledkem z bodu 1 a s výsledkem z Numpy.


In [23]:
def matrix_multiplication_2(A, B, C):
    for i in prange(A.shape[0]):
        for j in prange(B.shape[1]):
            for k in prange(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]

matrix_multiplication_2 = jit(matrix_multiplication_2)

In [24]:
C = np.zeros((1000, 1000))
matrix_multiplication_2(A, B, C)
print(np.allclose(C, C2))

True



3. Upravte funkci tak ať výpočet probíhá paralelně, otestujte jaký vliv má když jednotlivé z vnějších smyček nastavíte jako paralelní.
    - opět porovnejte výsledky s výsledkem z Numpy a s výsledkem z bodu 1 a 2.

In [25]:
def matrix_multiplication_3(A, B, C):
    chunk = 167
    for ic in prange(A.shape[0] // chunk + 1):
        for jc in prange(B.shape[1] // chunk + 1):
            for kc in prange(A.shape[1] // chunk + 1):
                for i in prange(ic * chunk, (ic+1) * chunk):
                    for j in prange(jc * chunk, (jc+1) * chunk):
                        for k in prange(kc * chunk, (kc+1)*chunk):
                            C[i, j] += A[i, k] * B[k, j]

matrix_multiplication_3 = jit(matrix_multiplication_3)

In [26]:
C = np.zeros((1000, 1000))
matrix_multiplication_3(A, B, C)
print(np.allclose(C, C2))

: 